In [1]:
import pandas as pd

In [2]:
INPUT_FILEPATH = "input/produccion_junio_2023.xlsx"
MES_A_ANALIZAR = "June"

In [3]:

# Este diccionario relaciona la hoja de los procedimientos a datos en el formato de SIGCOM.
# El primer elemento de la lista corresponde a la fila en donde debe ser cargado ese procedimiento.
# El segundo elemento corresonde a que CC de consulta debe ser cargado el procedimiento (en caso
# que tenga alguna consulta)
CORRELACION_PROCEDIMIENTOS_SIGCOM = {
    "QMT": ["PROCEDIMIENTO ONCOLOGÍA", "CONSULTA ONCOLOGÍA"],
    "APNEA": ["PROCEDIMIENTO DE NEUMOLOGÍA", "CONSULTA NEUMOLOGÍA"],
    "EF": ["PROCEDIMIENTOS DE CARDIOLOGÍA", ""],
    "EBUS": ["PROCEDIMIENTOS EBUS", "CONSULTA NEUMOLOGÍA"],
    "TAVI": ["PROCEDIMIENTOS TAVI", ""],
    "ASIST VENTRICULAR": ["PROCEDIMIENTOS ASISTENCIA VENTRICULAR", ""],
    "ECMO": ["PROCEDIMIENTOS ECMO", ""],
    "HMD": ["PROCEDIMIENTOS DE HEMODINAMIA", "CONSULTA CARDIOLOGÍA"],
    "BRONCOSCOPIA": ["PROCEDIMIENTOS ENDOSCÓPICOS", "CONSULTA NEUMOLOGÍA"],
}

SERVICIO_INTERNO_SIGCOM = {
    "ambulatorio": "CONSULTAS",
    "medico quirugico 3° norte y sur": "HOSPITALIZACIÓN MEDICINA INTERNA/HOSPITALIZACIÓN QUIRÚRGICA",
    "medico quirugico 4° norte": "HOSPITALIZACIÓN CARDIOVASCULAR",
    "uci 4° sur": "UNIDAD DE CUIDADOS INTENSIVOS",
    "uci 5° norte": "UNIDAD DE CUIDADOS INTENSIVOS",
    "uci 5° sur": "UNIDAD DE CUIDADOS INTENSIVOS",
    "uti 3° norte": "UNIDAD DE TRATAMIENTO INTENSIVO ADULTO",
    "uti 4° norte": "UNIDAD DE TRATAMIENTO INTENSIVO ADULTO",
    "uti 4° sur": "UNIDAD DE TRATAMIENTO INTENSIVO ADULTO",
    "uti 5° norte": "UNIDAD DE TRATAMIENTO INTENSIVO ADULTO",
}

In [4]:
def obtener_localizacion_de_procedimientos(ruta_produccion):
    resultados = {}
    archivo = pd.ExcelFile(ruta_produccion)
    for hoja in archivo.sheet_names[3:]:
        df = pd.read_excel(archivo, sheet_name=hoja)
        df = df.loc[:, "Rut":"Hoja1.Nombre nuevo HIS"]
        df["Hoja1.Nombre nuevo HIS"] = df["Hoja1.Nombre nuevo HIS"].str.strip().str.lower()
        df["SIGCOM"] = df["Hoja1.Nombre nuevo HIS"].replace(SERVICIO_INTERNO_SIGCOM)

        procedimientos_mes = df[df["Fecha Realización"].dt.month_name() == MES_A_ANALIZAR]
        procedimientos_por_servicio = (
            procedimientos_mes.groupby("SIGCOM")["Rut"]
            .count()
            .reset_index(name="cantidad_procedimientos")
        )

        consulta_acorde = CORRELACION_PROCEDIMIENTOS_SIGCOM[hoja][-1]
        dict_reemplazo_consulta = {"CONSULTAS": consulta_acorde}
        procedimientos_por_servicio["SIGCOM"] = procedimientos_por_servicio["SIGCOM"].replace(
            dict_reemplazo_consulta
        )

        procedimiento_acorde = CORRELACION_PROCEDIMIENTOS_SIGCOM[hoja][0]

        resultados[procedimiento_acorde] = procedimientos_por_servicio

    return resultados

In [5]:
resultados_procedimientos = obtener_localizacion_de_procedimientos(INPUT_FILEPATH)

In [11]:
formato = pd.read_excel("input/Formato_5_Distribución_Producción_CC_Apoyo.xlsx", header=2)
formato = formato.iloc[:, 1:]

In [13]:
formato

,Cod,Centro de Costo,Nro Producción,Tipo Producción,HOSPITALIZACIÓN MEDICINA INTERNA,HOSPITALIZACIÓN QUIRÚRGICA,HOSPITALIZACIÓN CARDIOVASCULAR,QUIRÓFANOS CARDIOVASCULAR,QUIRÓFANOS CIRUGÍA TORACICA,UNIDAD DE CUIDADOS INTENSIVOS,...,ECOCARDIOGRAFÍA,TOMOGRAFÍA,IMAGENOLOGÍA,ANATOMÍA PATOLÓGICA,BANCO DE SANGRE,SERVICIO FARMACEUTICO,CENTRAL DE ESTERILIZACIÓN,SERVICIO DE ALIMENTACIÓN,MANTENIMIENTO,ADMINISTRACIÓN
0,1001,PROCEDIMIENTO DE NEUMOLOGÍA,1,Procedimiento,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1002,PROCEDIMIENTOS ASISTENCIA VENTRICULAR,1,Procedimiento,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1004,PROCEDIMIENTOS DE CARDIOLOGÍA,1,Procedimiento,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1009,PROCEDIMIENTOS DE HEMODINAMIA,1,Procedimiento,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1015,PROCEDIMIENTO ONCOLOGÍA,1,Procedimiento,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1017,PROCEDIMIENTOS DE TRAUMATOLOGÍA,1,Procedimiento,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1019,PROCEDIMIENTOS EBUS,1,Procedimiento,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1020,PROCEDIMIENTOS ECMO,1,Procedimiento,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1021,PROCEDIMIENTOS ENDOSCÓPICOS,1,Procedimiento,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1024,PROCEDIMIENTOS TAVI,1,Procedimiento,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
